## Generate t2t data

In [1]:
# Enable TF Eager execution
import tensorflow as tf
tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Generate data

In [3]:
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
import generate_problem # self-defined 'generate_problem.py' file in the same folder.

USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD'
TMP_DIR = './tmp' # Where data files from internet stored
DATA_LOC = './data' # Where pre-prcessed data is stored

# Init problem T2T object the generated training data
t2t_problem = generate_problem.OnlineRevewProjectUSYD(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd.32768.subwords


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd-unshuffled-train-00000-of-00009', './data/online_revew_project_usyd-unshuffled-train-00001-of-00009', './data/online_revew_project_usyd-unshuffled-train-00002-of-00009', './data/online_revew_project_usyd-unshuffled-train-00003-of-00009', './data/online_revew_project_usyd-unshuffled-train-00004-of-00009', './data/online_revew_project_usyd-unshuffled-train-00005-of-00009', './data/online_revew_project_usyd-unshuffled-train-00006-of-00009', './data/online_revew_project_usyd-unshuffled-train-00007-of-00009', './data/online_revew_project_usyd-unshuffled-train-00008-of-00009', './data/online_revew_project_usyd-unshuffled-dev-00000-of-00001']
INFO:tensorflow:Skipping shuffle because output files exist


In [9]:
from tensor2tensor.utils import registry
from tensor2tensor import models
MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files'

In [10]:
from tensor2tensor.utils.trainer_lib import create_hparams
import json
# Init Hparams object from T2T Problem
hparams = create_hparams(HPARAMS)
hparams.batch_size = 512
# hparams.learning_rate_warmup_steps = 45000
# hparams.learning_rate = .4

# Can see all Hparams with code below
print(json.loads(hparams.to_json()))

{'video_num_input_frames': 1, 'use_fixed_batch_size': False, 'kernel_width': 1, 'multiproblem_mixing_schedule': 'constant', 'moe_hidden_sizes': '2048', 'optimizer_zero_grads': False, 'unidirectional_encoder': False, 'pretrained_model_dir': '', 'batch_size': 512, 'multiproblem_max_input_length': -1, 'proximity_bias': False, 'attention_value_channels': 0, 'optimizer_adam_epsilon': 1e-09, 'eval_drop_long_sequences': False, 'prepend_mode': 'prepend_inputs_masked_attention', 'eval_run_autoregressive': False, 'min_length': 0, 'optimizer_adafactor_clipping_threshold': 1.0, 'kernel_height': 3, 'attention_dropout': 0.1, 'length_bucket_step': 1.1, 'multiproblem_reweight_label_loss': False, 'parameter_attention_value_channels': 0, 'layer_prepostprocess_dropout_broadcast_dims': '', 'use_target_space_embedding': True, 'optimizer_adafactor_beta2': 0.999, 'filter_size': 2048, 'summarize_grads': False, 'moe_overhead_train': 1.0, 'ffn_layer': 'dense_relu_dense', 'max_input_seq_length': 0, 'scheduled_sa

In [8]:
from tensor2tensor.utils.trainer_lib import create_run_config, create_experiment

# running time calculation
import timeit
start = timeit.default_timer()

# Initi Run COnfig for Model Training
RUN_CONFIG = create_run_config(
    model_name=MODEL,# Location of where model file is store
    model_dir=TRAIN_DIR
      # More Params here in this fucntion for controling how noften to tave checkpoints and more. 
)

# Create Tensorflow Experiment Object
tensorflow_exp_fn = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=MODEL,
        problem_name= t2t_problem, #PROBLEM,
#         t2t_usr_dir='./',
        data_dir='./data', 
        train_steps=40000, # Total number of train steps for all Epochs
        eval_steps=200 # Number of steps to perform for each evaluation
    )

# Kick off Training
tensorflow_exp_fn.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()s
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_environment': 'local', 'use_tpu': False, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_eval_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f245f82b9e8>, '_task_id': 0, '_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 0, 't2

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


In [ ]:
# ~4h

# 2. using encoder and decoder

In [43]:
# Generate and view the data
# This cell is commented out because WMT data generation can take hours

# ende_problem.generate_data(data_dir, tmp_dir)
example = tfe.Iterator(t2t_problem.dataset(Modes.TRAIN, DATA_LOC)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.



INFO:tensorflow:Reading data files from ./data/online_revew_project_usyd-train*
INFO:tensorflow:partition: 0 num_data_files: 9


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [44]:

# Example inputs as int-tensor.
print("Inputs, encoded:")
print(inputs)
print("Inputs, decoded:")
# Example inputs as a sentence.
print(decode(inputs))

print("---"*10)
# Example targets as int-tensor.
print("Targets, encoded:")
print(targets)
# Example targets as a sentence.
print("Targets, decoded:")
print(decode(targets))

Inputs, encoded:
[6832, 10831, 4, 6832, 377, 2569, 7601, 2461, 26, 5, 3841, 1]
Inputs, decoded:
integers [6832, 10831, 4, 6832, 377, 2569, 7601, 2461, 26, 5, 3841]
targets <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fcc845c6128>
disposable razors and disposable 6 razor scam blades are a trap
------------------------------
Targets, encoded:
[75, 216, 1445, 3, 4, 26, 30, 2040, 10, 15, 74, 9, 2, 3260, 122, 5, 2159, 2569, 250, 5614, 3, 2, 579, 8, 145, 24482, 18562, 652, 6832, 2461, 9, 7557, 10, 4, 34, 1780, 10, 139, 110, 6, 75, 2461, 4, 5, 2159, 2569, 10, 59, 561, 4, 1177, 27, 31, 647, 17, 1]
Targets, decoded:
integers [75, 216, 1445, 3, 4, 26, 30, 2040, 10, 15, 74, 9, 2, 3260, 122, 5, 2159, 2569, 250, 5614, 3, 2, 579, 8, 145, 24482, 18562, 652, 6832, 2461, 9, 7557, 10, 4, 34, 1780, 10, 139, 110, 6, 75, 2461, 4, 5, 2159, 2569, 10, 59, 561, 4, 1177, 27, 31, 647, 17]
targets <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fcc845c612

In [34]:
hparams.problem_hparams

HParams([('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7fcc8c2db8d0>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7fcc8c2db908>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fcc8ccceac8>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fcc8ccceac8>}), ('was_copy', False), ('was_reversed', True)])

In [39]:
ende_problem = t2t_problem
vocab_name = "vocab.online_revew_project_usyd.32768.subwords"

# Get the encoders from the problem
DATA_LOC = './data'
encoders = ende_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    print("inputs",inputs)
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
        print("integers", integers)
        print("targets", encoders["targets"])
    return encoders["targets"].decode(np.squeeze(integers))

In [40]:
# Create hparams and the model
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
import json

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
hparams.problem_hparams.was_reversed =False

model = registry.model(MODEL)(hparams, Modes.EVAL) #tensorflow_exp_fn

In [41]:
ckpt_path = tf.train.latest_checkpoint("./model_files/") #model.ckpt
ckpt_path

'./model_files/model.ckpt-40000'

In [4]:
# import tensorflow as tf
# latest = tf.train.latest_checkpoint('./model_files/')
# latest

'./model_files/model.ckpt-40000'

In [42]:
import numpy as np


#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)

# inputs = '''
# the archetypical american novel features a solitary protagonist undertaking an odyssey in which heshe confronts both physical and moral challenges and through which heshe emerges with a renewed spirit, transformed by the crucible of confrontations with adversaries, both real and imagined  sara gruens engaging water for elephants is an eminently american work, set in the depths of the depression and featuring a brokenhearted young man whose unplanned existential leap of faith catapults him into a chaotic and unknown way of life
# '''
inputs = '''
happy with purchase even though it came a lot later than expected.'''
outputs = summarize(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

inputs [35548, 3852, 35554, 25, 306, 18, 347, 89, 196, 11, 271, 5, 165, 483, 73, 537, 35548, 9301, 35554, 25, 1]
integers [193, 3852, 193, 25, 306, 18, 347, 89, 196, 11, 271, 5, 165, 483, 73, 537]
targets <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fcc845c6128>
Inputs: 
happy with purchase even though it came a lot later than expected.
Outputs: each 10each happy with purchase even though it came a lot later than expected


In [18]:

#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)

inputs = '''
the archetypical american novel features a solitary protagonist undertaking an odyssey in which heshe confronts both physical and moral challenges and through which heshe emerges with a renewed spirit, transformed by the crucible of confrontations with adversaries, both real and imagined  sara gruens engaging water for elephants is an eminently american work, set in the depths of the depression and featuring a brokenhearted young man whose unplanned existential leap of faith catapults him into a chaotic and unknown way of life
'''

outputs = summarize(inputs)

print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)


Inputs: 
the archetypical american novel features a solitary protagonist undertaking an odyssey in which heshe confronts both physical and moral challenges and through which heshe emerges with a renewed spirit, transformed by the crucible of confrontations with adversaries, both real and imagined  sara gruens engaging water for elephants is an eminently american work, set in the depths of the depression and featuring a brokenhearted young man whose unplanned existential leap of faith catapults him into a chaotic and unknown way of life

Outputs: when 10so the archetypical american novel features a solitary protagonist undertaking an odyssey in which heshe styboth physical and moral challenges and through which heshe cost with a renewed spirit, transformed by the his his ble of she ations ies, both real real and imagined  sara gruens engaging water for elephants is an


In [ ]:
def continuous_decode_on_eval_data(self):
    """Decode from dataset on new checkpoint."""
    if self._hparams.mlperf_mode:
      ckpt_generator = next_undecoded_checkpoint(
          self._hparams.model_dir, self._decode_hparams.decode_timeout_mins)
    else:
      ckpt_generator = next_checkpoint(self._hparams.model_dir,
                                       self._decode_hparams.decode_timeout_mins)

    for ckpt in ckpt_generator:
      current_step = decoding.get_step_from_ckpt_path(ckpt)
      tf.logging.info("Decoding step %d" % current_step)
      # Skip checkpoint 0.
      if current_step == 0:
        continue
      # Decode the latest checkpoint by default.
      checkpoint_path = None
      if self._hparams.mlperf_mode:
        self._decode_hparams.mlperf_decode_step = current_step
        checkpoint_path = ckpt

      mlperf_log.transformer_print(key=mlperf_log.EVAL_START)
      self.decode(
          dataset_split=tf.estimator.ModeKeys.EVAL,
          checkpoint_path=checkpoint_path)
      d_hparams = self._decode_hparams
      if self._hparams.mlperf_mode and d_hparams.mlperf_success:
        mlperf_log.transformer_print(
            key=mlperf_log.RUN_STOP, value={"success": "true"})
        break

    d_hparams = self._decode_hparams
    if self._hparams.mlperf_mode and not d_hparams.mlperf_success:
      mlperf_log.transformer_print(
          key=mlperf_log.RUN_STOP, value={"success": "false"})

  def continuous_decode_from_file(self):
    """Decode from file on new checkpoint."""
    for _ in next_checkpoint(self._hparams.model_dir,
                             self._decode_hparams.decode_timeout_mins):
      self.decode(decode_from_file=True)
